In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv('/kaggle/input/poland-cars-for-sale-dataset/Car_sale_ads.csv')

In [ ]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
nu_col = df.nunique()
print(nu_col)

In [ ]:
df.isnull().sum()

drop the columns in which there are a lot of NaN.

In [ ]:
df1=df.drop(['Vehicle_version', 'Vehicle_generation','CO2_emissions','Origin_country','First_owner','First_registration_date'], axis=1)

In [ ]:
nu_col1 = df1.nunique()
print(nu_col1)

In [ ]:
df1.isnull().sum()

In [ ]:
df1.dtypes

In [ ]:
import matplotlib as plt
import seaborn as sns

In [ ]:
df1['Mileage_km']=df1['Mileage_km'].fillna(df1['Mileage_km'].mean())
df1['Power_HP']=df1['Power_HP'].fillna(df1['Power_HP'].mean())
df1['Displacement_cm3']=df1['Displacement_cm3'].fillna(df1['Displacement_cm3'].mean())

In [ ]:
df1.corr()

In [ ]:
sns.heatmap(df1.corr())

In [ ]:
df.groupby('Price')['Index'].count().plot(figsize=(15,10))

In [ ]:
df.groupby('Condition')['Index'].count().plot.pie(figsize=(12,7),autopct="%1.1f%%")

In [ ]:
df.groupby('Vehicle_brand')['Index'].count().sort_values(ascending=False).head(50).plot.bar(figsize=(15,10))

In [ ]:
df.groupby('Vehicle_model')['Index'].count().sort_values(ascending=False).head(50).plot.bar(figsize=(15,10))

In [ ]:
df.groupby('Production_year')['Index'].count().plot.bar(figsize=(15,10))

In [ ]:
df.groupby('Mileage_km')['Index'].count().plot(figsize=(15,10))

In [ ]:
df.groupby('Power_HP')['Index'].count().plot(figsize=(15,10))

In [ ]:
df.groupby('Displacement_cm3')['Index'].count().plot(figsize=(15,10))

In [ ]:
df.groupby('Fuel_type')['Index'].count().plot.pie(figsize=(12,7),autopct="%1.1f%%")

In [ ]:
df.groupby('Drive')['Index'].count().plot.pie(figsize=(12,7),autopct="%1.1f%%")

In [ ]:
df.groupby('Transmission')['Index'].count().plot.pie(figsize=(12,7),autopct="%1.1f%%")

In [ ]:
df.groupby('Type')['Index'].count().plot.pie(figsize=(12,7),autopct="%1.1f%%")

In [ ]:
df.groupby('Doors_number')['Index'].count().plot.pie(figsize=(12,7),autopct="%1.1f%%")

In [ ]:
df.groupby('Colour')['Index'].count().plot.pie(figsize=(12,7),autopct="%1.1f%%")

In [ ]:
nu_col1 = df1.nunique()
print(nu_col1)

In [ ]:
df1.describe()

In [ ]:
df1.groupby('Currency').count()

In [ ]:
df1=df1[df1['Currency']=='PLN']

Make Brand Popularity from Vehcle Brands

In [ ]:
df2=pd.DataFrame(df1['Vehicle_brand'].value_counts())

In [ ]:
df2

In [ ]:
df2['index']=df2.index

In [ ]:
df2

In [ ]:
df2 = df2.rename(columns={'Vehicle_brand': 'num','index':'Vehicle_brand'})

In [ ]:
df2

In [ ]:
df1=pd.merge(df1, df2, how='left', on ='Vehicle_brand')

In [ ]:
def rank(ex):
    if ex<=100:
        return 0
    elif 100<ex<=500:
        return 1
    elif 500<ex<=1000:
        return 2
    elif 1000<ex<=2000:
        return 3
    elif 2000<ex<=5000:
        return 4
    else:
        return 5

In [ ]:
df1.loc[:,'Brand_popularity']=df1['num'].apply(rank)

In [ ]:
df1.groupby('Brand_popularity')['Index'].count().plot.bar(figsize=(15,10))

Search the correlation between Price and Brand Popularity

In [ ]:
df1.corr()

I can find that 'Brand Popularity' has no correlation with price.

In [ ]:
df3=df1.drop(['Currency','Vehicle_brand','Vehicle_model','Offer_publication_date','Offer_location','Features','num'], axis=1)

In [ ]:
df3

In [ ]:
df3=pd.get_dummies(df3)

In [ ]:
df3

In [ ]:
df3.isnull().sum()

In [ ]:
df3=df3.drop('Index', axis=1)

In [ ]:
df3=df3.drop('Doors_number', axis=1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
sns.heatmap(df3.corr())

Prediction Model by LR, Lasso and Ridge

In [ ]:
from sklearn.linear_model import Ridge,Lasso,LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [ ]:
x=df3.drop(['Price'], axis=1)
y=df3['Price']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x, y,train_size=0.8,random_state=123)

In [ ]:
scaler=StandardScaler()
x_train_sc=pd.DataFrame(scaler.fit_transform(x_train))
x_test_sc=pd.DataFrame(scaler.transform(x_test))

In [ ]:
model1=LinearRegression()
model2=Lasso(alpha=1.0)
model3=Ridge(alpha=0.1)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train_sc):
    tr_x,va_x=x_train_sc.iloc[tr_idx],x_train_sc.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model1.fit(tr_x,tr_y)
    va_pred1=model1.predict(va_x)
    score_rmse1=np.sqrt(mean_squared_error(va_y,va_pred1)).mean()
    score_mae1=mean_absolute_error(va_y,va_pred1).mean()
    score_r21=r2_score(va_y,va_pred1).mean()

print('rmse1:',score_rmse1)
print('mae1:',score_mae1)
print('R21:',score_r21)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train_sc.iloc[tr_idx],x_train_sc.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model2.fit(tr_x,tr_y)
    va_pred2=model2.predict(va_x)
    score_rmse2=np.sqrt(mean_squared_error(va_y,va_pred2)).mean()
    score_mae2=mean_absolute_error(va_y,va_pred2).mean()
    score_r22=r2_score(va_y,va_pred2).mean()

print('rmse2:',score_rmse2)
print('mae2:',score_mae2)
print('R22:',score_r22)

In [ ]:
scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train_sc.iloc[tr_idx],x_train_sc.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    
    model3.fit(tr_x,tr_y)
    va_pred3=model3.predict(va_x)
    score_rmse3=np.sqrt(mean_squared_error(va_y,va_pred3)).mean()
    score_mae3=mean_absolute_error(va_y,va_pred3).mean()
    score_r23=r2_score(va_y,va_pred3).mean()

print('rmse3:',score_rmse3)
print('mae3:',score_mae3)
print('R23:',score_r23)

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test)

In [ ]:
params={'metric': 'rmse'}

In [ ]:
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=1000,
                early_stopping_rounds=100,
                verbose_eval=100)

In [ ]:
predicted = gbm.predict(x_test)

In [ ]:
def pred_map(pred_df):
    rmse = np.sqrt(mean_squared_error(pred_df['data'], pred_df['pred']))
    R2 = r2_score(pred_df['data'], pred_df['pred']) 
    plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    ax.scatter('data', 'pred', data=pred_df)
    ax.set_xlabel('Data Value', fontsize=10)
    ax.set_ylabel('Pred Value', fontsize=10)
    ax.set_xlim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    ax.set_ylim(pred_df.min().min()-0.1 , pred_df.max().max()+0.1)
    x = np.linspace(pred_df.min().min()-0.1, pred_df.max().max()+0.1, 2)
    y = x
    ax.plot(x,y,'r-')
    plt.text(0.1, 0.9, 'rmse = {}'.format(str(round(rmse, 5))), transform=ax.transAxes, fontsize=10)
    plt.text(0.1, 0.8, 'R2 = {}'.format(str(round(R2, 5))), transform=ax.transAxes, fontsize=10)

In [ ]:
df_prediction = pd.concat([y_test.reset_index(drop=True), pd.Series(predicted)], axis=1)
df_prediction.columns = ['data', 'pred']

In [ ]:
pred_map(df_prediction)

I got rmse  22629.94516 and R2 0.93035
with LightGBM without tuning